# Gather Dataset

In [62]:
from icrawler.builtin import GoogleImageCrawler
import datetime
def datetime2tuple(date):
    return (date.year, date.month, date.day)
def gather_data(n_total_images,n_per_crawl,keyword, folder_name):
    delta = datetime.timedelta(days=30)
    end_day = datetime.datetime(2023, 3, 17)
    for i in range(int(n_total_images / n_per_crawl )):
        start_day = end_day - delta
        google_crawler = GoogleImageCrawler(downloader_threads=4, storage={'root_dir': folder_name})
        google_crawler.crawl(keyword=keyword[i], filters={'date':(datetime2tuple(start_day), datetime2tuple(end_day))}, file_idx_offset=i*n_per_crawl , max_num=n_per_crawl)
        end_day = start_day - datetime.timedelta(days=1)

#gather_data(700,100,['packages ready to ship','packages about to be sent','cardboard boxes','packages','box packages','sealed packages','delivery cardboard package box'],'goodpackages')
#gather_data(200,100,['ripped cardboard boxes','ripped packages'],'rippedpackages') #ripped
#gather_data(200,100,['destroyed packages','damaged cardboard boxes'],'crushedpackages') #crushed
#gather_data(200,100,['cardboard boxes with water damage','wet cardboard boxes'],'leakingpackages')#leaking
#gather_data(200,100,['burned cardboard','charred cardboard'],'burnedpackages') # burned
#gather_data(200,100,['cardboard recycling pile','broken down cardboard boxes'],'foldedpackages')#folded

# Split the Data

In [63]:
import splitfolders
import shutil
import os
input_folder = "data\startingdata\\"
output_folder = "data\\split data\\"
splitfolders.ratio(input_folder, output=output_folder,
                   seed=42, ratio=(.8, 0, .2))
subsets = ['test\\', 'train\\']
for subset in subsets:
    output = output_folder+subset
    destination = "data\\split data\\"+subset+"\\badpackages\\"
    categories = ['burnedpackages', 'crushedpackages',
                  'foldedpackages', 'leakingpackages', 'rippedpackages']
    for category in categories:
        source_folder = output + category
        for file_name in os.listdir(source_folder):

            if os.path.exists(destination + '\\' + file_name):
                data = os.path.splitext(file_name)
                only_name = data[0]
                extension = data[1]
                new_base = category + only_name + extension
                new_name = os.path.join(destination, new_base)
                shutil.move(source_folder + '\\' + file_name, new_name)
            else:
                shutil.move(source_folder + '\\' + file_name,
                            destination + '\\' + file_name)
        os.rmdir(source_folder)


Copying files: 1085 files [00:01, 813.42 files/s]


# Formatting the Data

In [64]:
from tensorflow.keras.applications.resnet50 import preprocess_input,decode_predictions,ResNet50
from tensorflow.keras.layers import Conv2D,Dense, Flatten,MaxPool2D,BatchNormalization,GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential,Model
import matplotlib.pyplot as plt
import numpy

img_height,img_width = (224,224)
batch_size = 32

train_data_path = "data/split data/train/"
test_data_path = "data/split data/test/"

# Prep the training data

In [65]:
training_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,shear_range=.2,zoom_range=0.2,horizontal_flip=True,validation_split=0.4) #might not need validation split
training_data_generator = training_datagen.flow_from_directory(train_data_path,target_size=(img_height,img_width),batch_size=batch_size,class_mode='categorical',subset='training')
testing_data_generator = training_datagen.flow_from_directory(test_data_path,target_size=(img_height,img_width),batch_size=1,class_mode='categorical',subset='validation')


Found 520 images belonging to 2 classes.
Found 87 images belonging to 2 classes.


In [66]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
base_model = ResNet50(include_top=False,weights='imagenet')
output=base_model.output
output=GlobalAveragePooling2D()(output)
output=Dense(1024,activation='relu')(output)
predictions = Dense(training_data_generator.num_classes,activation='softmax')(output)
model = Model(inputs=base_model.input,outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.fit(training_data_generator,epochs=10)


Epoch 1/10


c:\Users\willi\anaconda3\lib\site-packages\PIL\Image.py:945: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


17/17 [==============================] - 32s 2s/step - loss: 1.3760 - accuracy: 0.6827
Epoch 2/10
17/17 [==============================] - 28s 2s/step - loss: 0.6508 - accuracy: 0.7769
Epoch 3/10
17/17 [==============================] - 28s 2s/step - loss: 0.3468 - accuracy: 0.8346
Epoch 4/10
17/17 [==============================] - 29s 2s/step - loss: 0.2308 - accuracy: 0.9154
Epoch 5/10
17/17 [==============================] - 29s 2s/step - loss: 0.2133 - accuracy: 0.9077
Epoch 6/10
17/17 [==============================] - 28s 2s/step - loss: 0.1538 - accuracy: 0.9308
Epoch 7/10
17/17 [==============================] - 31s 2s/step - loss: 0.1622 - accuracy: 0.9385
Epoch 8/10
17/17 [==============================] - 30s 2s/step - loss: 0.1021 - accuracy: 0.9615
Epoch 9/10
17/17 [==============================] - 29s 2s/step - loss: 0.0827 - accuracy: 0.9731
Epoch 10/10
17/17 [==============================] - 27s 2s/step - loss: 0.0862 - accuracy: 0.9692


In [67]:
model.save('Models\\Resnet50\\Resnet50_with_50_10_datasplit.h5')

# Evaluation

In [68]:
test_loss,test_accuracy = model.evaluate(testing_data_generator,verbose=2)
print(test_accuracy)

87/87 - 8s - loss: 0.5356 - accuracy: 0.7931 - 8s/epoch - 88ms/step
0.7931034564971924
